In [20]:
import pandas as pd
import kaggle
import pickle

import os
path=r"C:\Users\berid\python\yts mx project"

In [5]:
kaggle.api.dataset_download_files('beridzeg45/all-movies-on-imdb',path=os.path.join(path,'scraped_data'),unzip=True,quiet=False)

Dataset URL: https://www.kaggle.com/datasets/beridzeg45/all-movies-on-imdb


100%|██████████| 28.0M/28.0M [00:05<00:00, 5.08MB/s]


In [6]:
df=pd.read_csv(os.path.join(path,'scraped_data','movies.csv'))

In [12]:
df.sample()

,Movie URL,Title,Poster,Release Year,Length in Min,IMDB Rating,Rating Count,Plot,Directors,Writers,Stars,Genres
88296,https://www.imdb.com/title/tt0920473/?ref_=sr_...,Un baiser s'il vous plaît,https://m.media-amazon.com/images/M/MV5BMTY0OD...,2007.0,96.0,6.7,2700.0,When Gabriel and Emilie meet totally by chance...,Emmanuel Mouret,Emmanuel Mouret|,Julie Gayet|Michaël Cohen|Virginie Ledoyen,Comedy|Romance


In [13]:
df['Movie URL']=df['Movie URL'].str.split('?').str[0]

In [28]:
usefull_movie_urls=pd.read_csv(os.path.join(path,'scraped_data','cleaned_yts_movies_df.csv'))['IMDB_LINK'].unique()

df=df[df['Movie URL'].isin(usefull_movie_urls)]

### Bigrams

In [30]:
import nltk
from nltk.util import ngrams
from collections import Counter
import contractions

from nltk.sentiment import SentimentIntensityAnalyzer
sia=SentimentIntensityAnalyzer()

from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

from nltk.corpus import stopwords
stopwords=stopwords.words('english')
stopwords_extended = stopwords+['a', 'the', 'but', 'and', 's', '.', ',', "'s", 'i', 'in', 'to',':',';',"'",'(',')','–']


In [33]:
def get_bigrams(text):
    try:
        text = contractions.fix(text)
        plot_tokenized = nltk.word_tokenize(text)
        tokens_lowered = [t.lower().strip() for t in plot_tokenized]
        removed_stopwords = [t for t in tokens_lowered if t not in stopwords_extended]
        return list(ngrams(removed_stopwords, 2))
    except:
        return 'No Plot'

# Extract bigrams for all plots and count them
all_bigrams = []
df['Plot'].apply(lambda plot: all_bigrams.extend(get_bigrams(plot)))
df['BIGRAMS']=df['Plot'].apply(lambda plot:get_bigrams(plot))

# Count the most common bigrams
bigram_counts = Counter(all_bigrams)
most_common_bigrams = bigram_counts.most_common(10000)
most_common_bigrams=[i[0] for i in most_common_bigrams]

C:\Users\berid\AppData\Local\Temp\ipykernel_11632\508433305.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BIGRAMS']=df['Plot'].apply(lambda plot:get_bigrams(plot))


In [34]:
bigrams_series=[]
for i,bigrams_list in enumerate(df['BIGRAMS'],start=1):
    new_bigrams_list=[]
    for bigram in bigrams_list:
        if bigram in most_common_bigrams:
            new_bigrams_list.append(bigram)
    bigrams_series.append(new_bigrams_list)
    
    print(f'{i}/{len(df['BIGRAMS'])}',end='\r')

In [35]:
df['BIGRAMS']=bigrams_series

C:\Users\berid\AppData\Local\Temp\ipykernel_11632\3023807138.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BIGRAMS']=bigrams_series


In [36]:
bigram_texts_series=[]

for i,bigrams_list in enumerate(df['BIGRAMS'],start=1):
    bigram_texts_list=[]
    for bigram in bigrams_list:
        bigram_text=' '.join([w for w in bigram])
        bigram_texts_list.append(bigram_text)
    
    bigram_texts_list='|'.join(i for i in bigram_texts_list)
    bigram_texts_series.append(bigram_texts_list)

df['ALL_TAGS']=bigram_texts_series
df=df.drop(columns='BIGRAMS')

C:\Users\berid\AppData\Local\Temp\ipykernel_11632\4025775031.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ALL_TAGS']=bigram_texts_series


In [37]:
df=df[['Movie URL', 'ALL_TAGS']]

In [38]:
df.to_csv(os.path.join(path,'scraped_data','cleaned_imdb_movies_df.csv'),index=False)

In [40]:
try:
    os.remove(os.path.join(path,'scraped_data','movies.csv'))
except:
    print('No Movies.csv')